In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost/companies')
db = client.companies
col = db.geo_ref
col.find({})

In [3]:
list(col.find({'venue_location.PointCategory': '4square location'}).limit(4))

[]

In [4]:
list(col.find({"venue_location.GeoPoint":  {
                                        "$near": { 
                                            "$geometry": {
                                                  "type": "Point" ,
                                                  "coordinates": [ -95.712891, 37.09024 ]
                                            },
                                        }
                                        }}, {"venue_location.GeoPoint":1}).limit(3))

[{'_id': ObjectId('5e9d8c611d2a8adfeae25382'),
  'venue_location': {'GeoPoint': {'type': 'Point',
    'coordinates': [-95.712891, 37.09024]}}},
 {'_id': ObjectId('5e9d8c6fb6b1202989bf2614'),
  'venue_location': {'GeoPoint': {'type': 'Point',
    'coordinates': [-95.7081298828125, 37.09497833251953]}}},
 {'_id': ObjectId('5e9d8c6fb6b1202989bf2617'),
  'venue_location': {'GeoPoint': {'type': 'Point',
    'coordinates': [-95.71091651916504, 37.05617031622711]}}}]

# Use the $near operator to find airports near the companies

In [10]:
# Queary a cursor only with offices 
offices = list(col.find({"venue_location.PointCategory":"business office"},
                        {"venue_location.GeoPoint":1}).limit(10))

# Define the specific categories in an Array
filter_categories = ['Coffee Shop', 'Heliport', "Airport Terminal"]

def evaluateOfficeNeighborhood(document, filter_categories, radius):
    # Take office-document's coordinates
    print(f"\nEvaluating this office location: {document['venue_location']['GeoPoint']['coordinates']}")
    office_coords = document['venue_location']['GeoPoint']['coordinates']

    # Perform a query that finds nearby venues which are part of the `filter_categories`
    matches = list(col.find({"venue_location.GeoPoint":  {
                                        "$near": { 
                                            "$geometry": {
                                                  "type": "Point" ,
                                                  "coordinates": office_coords
                                            },
                                            "$maxDistance":radius,
                                           
                                        }
                                        },
                        'venue_location.PointCategory.categories_names': {"$in": filter_categories},
                        },
                        {"venue_location.PointCategory.categories_names":1}))
    
    # Loop the resulting cursor, and try to return the category found
    for venue in matches:
            yield venue['venue_location']['PointCategory']['categories_names']
            
for document in offices:
    venues = list(evaluateOfficeNeighborhood(document, filter_categories, 50000))
    print(venues)
    
    for var in filter_categories:
        print(f" There are {venues.count(var)} '{var}'")


Evaluating this office location: [-95.712891, 37.09024]
['Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop']
 There are 5 'Coffee Shop'
 There are 0 'Heliport'
 There are 0 'Airport Terminal'

Evaluating this office location: [-73.985506, 40.757929]
['Coffee Shop', 'Coffee Shop', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Coffee Shop', 'Airport Terminal', 'Airport Terminal', 'Heliport', 'Heliport', 'Heliport', 'Heliport', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal', 'Heliport', 'Heliport', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal', 'Airport Terminal']
 There are 12 'Coffee Shop'
 There are 20 'Heliport'
 There are 1

In [6]:
# Count how many airports and heliports are near the queried office

In [19]:
# Query a cursor only with offices 
offices = list(col.find({"venue_location.PointCategory":"business office"},
                        {"venue_location.GeoPoint":1}).limit(10))

# Define the specific categories in an Array
coffee_venues = ['Coffee Shop']
transport_venues = ['Heliport', "Airport Terminal"]
BigPicture = [[coffee_venues,500], # Coffee shops in a 500 mts radius
              [transport_venues,20000]] # Airports in a 20km radius

def exploreNeighborhood(document, filter_categories, radius):
    # Take office-document's coordinates
    office_coords = document['venue_location']['GeoPoint']['coordinates']
    print(f"\nEvaluating this office location: {office_coords}")
  
    # Query and make a list of matches. 
    # This will include nearby venues 
    #      - which are part of the `filter_categories` list
    #      - which are in the specified radius
    matches = list(col.find({"venue_location.GeoPoint":  {
                                        "$near": { 
                                            "$geometry": {
                                                  "type": "Point" ,
                                                  "coordinates": office_coords
                                            },
                                            # Specify the maximum radius of the query
                                            "$maxDistance":radius,
                                        }
                            },
                        # Specify the correct 'categories_names' values
                        'venue_location.PointCategory.categories_names': {"$in": filter_categories},
                        },
                        #
                        {"venue_location.PointCategory.categories_names":1}))
    
    # Loop the resulting cursor, and try to return the category found
    for venue in matches:
            yield venue['venue_location']['PointCategory']['categories_names']

def evaluateNeighborhood(BigPicture):
    for filter_categories, radius in BigPicture:
        for document in offices:
            venues = list(exploreNeighborhood(document, filter_categories, 50000))
            #print(venues)

            for var in filter_categories:
                print(f" There are {venues.count(var)} '{var}'")
                
evaluateNeighborhood(BigPicture)


Evaluating this office location: [-95.712891, 37.09024]
 There are 5 'Coffee Shop'

Evaluating this office location: [-73.985506, 40.757929]
 There are 12 'Coffee Shop'

Evaluating this office location: [-73.985506, 40.757929]
 There are 12 'Coffee Shop'

Evaluating this office location: [-73.985506, 40.757929]
 There are 12 'Coffee Shop'

Evaluating this office location: [-73.985506, 40.757929]
 There are 12 'Coffee Shop'

Evaluating this office location: [-73.985506, 40.757929]
 There are 12 'Coffee Shop'

Evaluating this office location: [-111.9035, 33.8171]
 There are 0 'Coffee Shop'

Evaluating this office location: [13.4109071, 52.5013449]
 There are 0 'Coffee Shop'

Evaluating this office location: [-111.9035, 33.8171]
 There are 0 'Coffee Shop'

Evaluating this office location: [-111.9035, 33.8171]
 There are 0 'Coffee Shop'

Evaluating this office location: [-95.712891, 37.09024]
 There are 0 'Heliport'
 There are 0 'Airport Terminal'

Evaluating this office location: [-73.98

In [7]:
0/0


ZeroDivisionError: division by zero

In [ ]:
list(col.find({"venue_location.GeoPoint":  {
                                        "$near": { 
                                            "$geometry": {
                                                  "type": "Point" ,
                                                  "coordinates": [ -95.712891, 37.09024 ]
                                            },
                                        }
                                        }}, {"venue_location.GeoPoint":1}).limit(3))